# Import Libraries

In [319]:
import gurobipy as grb
import pandas as pd
from gurobipy import GRB
import numpy as np

# Model

In [320]:
 # Create a model
model = grb.Model("Model")

# Get data from json

In [321]:
# get the  data from the json file small.json
import json
with open('small.json') as f:
    data = json.load(f)
# data

## Constants from data

In [322]:
horizon= data['horizon']
qualifications=data['qualifications']
staff_names=[i['name'] for i in data['staff']]
staff_qualifications=[i['qualifications'] for i in data['staff']]
job_list=[i['name'] for i in data['jobs']]
jour_list=[i for i in range(1,horizon+1)]
job_penality=[i['daily_penalty'] for i in data['jobs']] 
job_qualifications= [i['working_days_per_qualification'] for i in data['jobs']]
job_gain=[i['gain'] for i in data['jobs']]
due_dates=[i['due_date'] for i in data['jobs']]
vacation_staff=[i['vacations'] for i in data['staff']]

# Model's parameters

##### Objective 1
H
Q
S
J

qi
qj
vi

njk

cj
gj
dj

yj
lj
ej

pi_j_k_t

##### Objective 2
aij : j assigned to i, binary

ni : number of jobs for i

max_i : max ni

In [323]:
H = [i for i in range(1,horizon+1)]
Q = qualifications
S = staff_names
J = job_list
# Parameters for each staff

qi = {}
for i in range(len(staff_names)) :
    k = staff_names[i]
    qi[k] = []
    for c in qualifications:
        if c in data['staff'][i]['qualifications']:
            qi[k].append(c)
vi={}
for ind,i in enumerate(staff_names):
    vi[i] = []
    for j in jour_list:
        if j in vacation_staff[ind]:
            vi[i].append(j)
#vi=staff_in_vacation.to_dict(orient='index')

# Parameters for each job
qj = dict()
for ind, j in enumerate(job_list):
    qj[j] = list(job_qualifications[ind].keys())
njk = dict(zip(job_list,job_qualifications))
cj=job_penality_dict=dict(zip(job_list,job_penality))
gj=job_gain_dict=dict(zip(job_list,job_gain))
dj = dict(zip(job_list,due_dates))


# Decision variables

In [324]:
# Add decision variables
pi_j_k_t = model.addVars(staff_names,job_list,qualifications,jour_list, vtype=GRB.BINARY, name="pi_k_j_c")

yj = model.addVars(job_list, vtype=GRB.BINARY, name="yj")
lj = model.addVars(job_list, vtype=GRB.INTEGER, name="lj")
ej = model.addVars(job_list, vtype=GRB.INTEGER, name="ej")

max_i = model.addVar(vtype=GRB.INTEGER, name="max_i")

ni = model.addVars(S, vtype=GRB.INTEGER, name="ni")

aij = model.addVars(S, J, vtype=GRB.BINARY, name="aij")

# Constraints

In [325]:
constr1 = {f'constr1{i}_{t}':model.addConstr( grb.quicksum(pi_j_k_t[i,j,k,t] for j in J for k in Q) <= 1 , name=f"constr1{i}_{t}")
  for i in S
  for t in H}

constr2 = {f'constr2{i}_{t}':model.addConstr( grb.quicksum(pi_j_k_t[i,j,k,t] for j in J for k in Q) == 0 , name=f"constr2{i}_{t}")
  for i in S
  for t in vi[i]}

constr3 = {f'constr3{i}_{j}_{k}_{t}':model.addConstr( pi_j_k_t[i,j,k,t] == 0 , name=f"constr3{i}_{j}_{k}_{t}")
  for i in S
  for j in J
  for k in Q if k not in qj[j] or k not in qi[i]
  for t in H}

constr4 = {f'constr4{j}_{k}': model.addConstr( yj[j]*njk[j][k] <= grb.quicksum(pi_j_k_t[i,j,k,t] for i in S for t in H) , name=f"constr4{j}_{k}")
  for j in J
  for k in qj[j]}

constr5 = {f'constr5{j}_{k}': model.addConstr( grb.quicksum(pi_j_k_t[i,j,k,t] for i in S for t in H) <= njk[j][k] , name=f"constr5{j}_{k}")
  for j in J
  for k in qj[j]}

constr6 = {f'constr6{i}_{j}_{k}_{t}':model.addConstr( pi_j_k_t[i,j,k,t]*t <= ej[j] , name=f"constr6{i}_{j}_{k}_{t}")
  for i in S
  for j in J
  for k in Q
  for t in H}

constr7 = {f'constr7{j}' : model.addConstr( ej[j] - dj[j] <= lj[j] , name =f"constr7{j}")
  for j in J}

constr8 = {f'constr8{j}' : model.addConstr( 1 <= ej[j] , name =f"constr8{j}")
  for j in J}

constr9 = {f'constr9{j}' : model.addConstr( ej[j] <= H[-1] , name =f"constr9{j}")
  for j in J}

In [326]:
# assigned jobs to a staff member i is aij[i, j]
constr10 = {f'constr10{i}_{j}_{k}_{t}' : model.addConstr( pi_j_k_t[i,j,k,t] <= aij[i, j] , name = f"constr10{i}_{j}_{k}_{t}")
    for i in S
    for j in J
    for k in Q
    for t in H}

# number of jobs assigned to a staff member i is ni[i]
constr11 = {f'constr11{i}' : model.addConstr( grb.quicksum(aij[i, j] for j in J) <= ni[i] , name = f"constr11{i}")
    for i in S}

# max_i max number of jobs assigned to a staff member
# for all staff i, number of jobs assigned to i is less than or equal to max_i
constr12 = {f'constr12{i}' : model.addConstr( ni[i] <= max_i , name = f"constr12{i}")
    for i in S}

# Objectives

In [327]:
model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) - max_i , GRB.MAXIMIZE)

# model.setObjectiveN( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) , 0, GRB.MAXIMIZE)
# model.setObjectiveN( max_i , 1, GRB.MINIMIZE)

# Paramétrage (mode mute)
model.params.outputflag = 0
# Résolution du PL
model.optimize()

In [328]:
#model.computeIIS()
model.write("model.lp")

In [329]:
model.ObjVal

63.0

In [330]:
values= []
for k, v in pi_j_k_t.items():
    values.append(v.x)

In [331]:
res = pd.DataFrame(list(pi_j_k_t.keys()))
res[4] = values
result = res[res[4] == 1]
result

,0,1,2,3,4
1,Olivia,Job1,A,2,1.0
7,Olivia,Job1,B,3,1.0
10,Olivia,Job1,C,1,1.0
73,Olivia,Job5,C,4,1.0
74,Olivia,Job5,C,5,1.0
108,Liam,Job3,A,4,1.0
126,Liam,Job4,B,2,1.0
127,Liam,Job4,B,3,1.0
190,Emma,Job3,C,1,1.0
193,Emma,Job3,C,4,1.0


## Result Table

In [332]:
def color_table(x):
    if pd.isna(x):
        return "background-color: white"
    else:
        if "Job1" in x :
            return "background-color: blue"
        elif "Job2" in x:
            return "background-color: pink"
        elif "Job3" in x:
            return "background-color: orange"
        elif "Job4" in x:
           return "background-color: yellow"
        elif "Job5" in x:
           return "background-color: green"
        else:
           return  "background-color: red"

In [333]:
df = pd.DataFrame(columns = [i for i in range(1,6)], index = staff_names)

for ind, val in result.iterrows():
    col = val[3]
    row = val[0]
    v = val[1] + " " + val[2]
    df.loc[row,col] = v
    # vacation
    for staff in staff_names:
        for day in jour_list:
            if day in vi[staff]:
                df.loc[staff,day] = 'X'
df.style.applymap(color_table)

,1,2,3,4,5
Olivia,Job1 C,Job1 A,Job1 B,Job5 C,Job5 C
Liam,X,Job4 B,Job4 B,Job3 A,nan
Emma,Job3 C,X,Job4 C,Job3 C,nan
